In [1]:
import numpy as np
import os
import pickle
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense, LSTM, Concatenate, LeakyReLU, Softmax, Dropout
from keras.layers import Lambda, Flatten, Bidirectional, TimeDistributed, Reshape, MultiHeadAttention, LayerNormalization
from keras.activations import tanh
from keras.models import Model, Sequential
import keras.backend
import random
from copy import copy
#import necessary notebooks
import import_ipynb
from Jobs_and_Machines import *
from States_and_Policies import *
from TransformerLayers import *

importing Jupyter notebook from Jobs_and_Machines.ipynb
importing Jupyter notebook from States_and_Policies.ipynb
importing Jupyter notebook from Global_Variables.ipynb
importing Jupyter notebook from TransformerLayers.ipynb


In [2]:
#change working directory
os.chdir('D:\\Job-Scheduling-Files')
os.getcwd()

'D:\\Job-Scheduling-Files'

In [3]:
n = 8
n_min = 3

m = 4
m_min = 2

DS_max = 1 #10
DS_min = 1

In [4]:
def load_data(DS_min,DS_max):
    
    data_dict = dict(((n_state,m_state),[[],[]]) 
                           for n_state in range(n_min,n+1) for m_state in range(m_min,m+1))
    
    #load data
    for n_state in range(n_min, n+1):
        for m_state in range(m_min, m+1):
            x_j = []
            x_m = []
            y = []
            for DS in ["0"*(2-len(str(i)))+str(i) for i in range(DS_min,DS_max+1)]:
                training_data_path = f'Data/DataSet_{DS}/LSTM_Data_{DS}/{n_state}-jobs-{m_state}-machines_{DS}.pickle' #LSTM instead of Strd
                with open(training_data_path, 'rb') as f:
                    data = pickle.load(f)
                    machines_info = data[0][0][:,:,:-2].reshape((-1,n_state,m_state,4))
                    jobs_info = data[0][0][:,:,-2:]
                    x_m.append(machines_info)
                    x_j.append(jobs_info)
                    y.append(data[1][0])
            data_dict[(n_state,m_state)][0].append(np.concatenate(x_m))
            data_dict[(n_state,m_state)][0].append(np.concatenate(x_j))
            data_dict[(n_state,m_state)][1].append(np.concatenate(y))
            
    return data_dict

In [5]:
training_dict = load_data(DS_min,DS_max)
validation_dict = load_data(98,98)
#test_dict = load_data(99,99)
test_dict = load_data(2,2)

In [6]:
#split data into input and target and bring input into comprehensible form for LSTM
x_train_list, y_train_list = [], []
x_val_list, y_val_list = [], []
x_test_list, y_test_list = [], []

for key in training_dict:
    #training data
    x_train, y_train = training_dict[key]
    x_train_list.append(x_train)
    #x_train_list.append(runtimes_as_sequence(x_train))
    #x_train_list.append(data_to_transformer_format(x_train))
    y_train_list.append(y_train)
    
    #validation data
    x_val, y_val = validation_dict[key]
    x_val_list.append(x_val)
    y_val_list.append(y_val)
    
    #test data
    x_test, y_test = test_dict[key]
    x_test_list.append(x_test)
    y_test_list.append(y_test)

In [7]:
"""Dropout muss noch hinzugefügt werden"""
inputs_machines = keras.Input(shape=(None,None,4), name="Machine_Info_with_Runtimes")
inputs_jobs = keras.Input(shape=(None,2), name="Jobs_Deadlines/Weights")

#FF on RT + Machine Info
x_m = FeedForward(8,4, name="FF_Machines_Embedding")(inputs_machines)
#Transformer-Encoder with Self-Attention
x_m = TimeDistributed(TransformerLayer(d_model=8,num_attention_heads=2,dff=8,self_att=True, name="Machines_for_every_JobRTs"), name="Encoder")(x_m)

#FF on Deadline and Weight of Jobs
x_j = FeedForward(8,4, name="FF_Jobs_Embedding")(inputs_jobs)
#Input Self-Attention of Transformer Decoder
x_j = TransformerLayer(d_model=8,num_attention_heads=2,dff=8,self_att=True, name="Decoder_Input")(x_j)

#Merge all the information for every Job to pass it to Transformer Decoder Attention Part
x_seq = Merge(name="Merge_every_Job_with_Machines")(x_j,x_m) #We obtain a sequence of jobs encoded and with the machine information already integrated

#Attention-Decoder for every Job
x_seq = TimeDistributed(TransformerLayer(d_model=8,num_attention_heads=2,dff=8,self_att=False, name="Job_on_Machines"), name="Decoder_Attention")(x_seq)

#Output Attention-Layer of Decoder
x_seq = TransformerLayer(d_model=8,num_attention_heads=2,dff=8,self_att=True, name="Decoder_Output")(x_seq)

#Add Zero-Line for Machine Turn-Off
x_seq = AddZeroLine(name="Add_Machine_Shut_Down")(x_seq)

#Pass Transformer-Output through LSTM
lstm_out = Bidirectional(LSTM(16, name="Combine_Outputs"), name="Bidir.LSTM")(x_seq) #return_sequences=True
#lstm_out = Bidirectional(LSTM(16))(lstm_out)

#Apply Final FF on both                #maybe we could add a residual connection and a NormLayer here like in the Transformer Model
val = FeedForward(16,16, name="Value_Embedding")(x_seq)
query = FeedForward(16,16, name="Query_Embedding")(lstm_out)

#Pointer
output = Pointer(32, name="Pointer-Attentions")(val,query)

"""
#Add Zero-Line for Machine Turn-Off
print(x.shape)
#zero_line = tf.zeros((x.shape[1],x.shape[2]))
zero_line = tf.zeros_like(x[:,0:1,:])
val = Concatenate(axis=1)([x,zero_line])
print(val.shape)
#Pass Transformer-Output through LSTM
query = Bidirectional(LSTM(16))(x)
#Apply Final FF on both                #maybe we could add a residual connection here like in the Transformer Model
val = FeedForward(16,16)(val)
query = FeedForward(16,16)(query)
#Apply Pointer-NN to Hidden states
output = Attention(32)(val,query)
"""

Transformer = keras.Model(inputs=[inputs_machines, inputs_jobs], outputs=output)


#Pass final output through FF for Value to turn-off-machine
 #...
#Apply Pointer-NN to Hidden states
 #...
#Pass them through FF
 #...
#Merge everything


#Possible things to add:
    #An own Weight matrix to find out the value for shutting-off the Machine
    #Transform LSTM to Encoder-Decoder with Attention like in that Paper

In [ ]:
Transformer.summary()

In [ ]:
keras.utils.plot_model(Transformer, 'Transformer.png', show_shapes=True)

In [8]:
def costs(y_true, y_pred):
    
    indices = keras.backend.argmax(y_pred, axis=1)
    length = np.shape(indices)[0]
    inv_values = keras.backend.eval(y_true)[np.arange(length),indices]
    Q_factors = 1/inv_values
    Q_factor = np.mean(Q_factors)
    
    Q_factor = (Q_factor - 1)*100
    Q_factor = round(Q_factor,2) #vllt mal 10.000 nehmen, int verwandeln und durch 100 teilen
    
    return Q_factor

In [9]:
def MSE_with_Softmax(y_pred, y_true):
    y_pred = Softmax()(y_pred)
    y_true = Softmax()(y_true)
    loss = keras.losses.MeanSquaredError()(y_pred,y_true)
    #return loss
    return (loss * 10e4)

In [10]:
learning_rate = 0.001

Transformer.compile(
        #loss = keras.losses.CategoricalCrossentropy(),
        #loss = keras.losses.MeanSquaredError(),
        loss = MSE_with_Softmax,
        optimizer = keras.optimizers.Adam(learning_rate = learning_rate),
        run_eagerly=True,
        metrics = [costs])

In [11]:
print("We train on ", len(x_train_list), " different n-m-combinations.")
#x_train_orig, y_train_orig = np.copy(x_train_list), np.copy(y_train_list)
ep=0
for i in range(30):#range(50)
    zipped_list = list(zip(x_train_list,y_train_list))
    random.shuffle(zipped_list)
    x_train_list, y_train_list = zip(*zipped_list)
    for j in range(len(x_train_list)):
        history = Transformer.fit(x_train_list[j], y_train_list[j], shuffle=True, batch_size=128000, epochs=ep+1, initial_epoch=ep) #validation_data=(x_val_list[j],y_val_list[j])) #callbacks=[my_val_callback]
        ep = history.epoch[-1]+1

We train on  18  different n-m-combinations.
1/1 [==============================] - 8s 8s/step - loss: 68.5085 - costs: 34.5200
Epoch 2/2
1/1 [==============================] - 11s 11s/step - loss: 65.9201 - costs: 38.4000
Epoch 3/3
1/1 [==============================] - 7s 7s/step - loss: 47.7322 - costs: 19.4300
Epoch 4/4
1/1 [==============================] - 6s 6s/step - loss: 132.7313 - costs: 38.1800
Epoch 5/5
1/1 [==============================] - 5s 5s/step - loss: 104.8291 - costs: 29.0000
Epoch 6/6
1/1 [==============================] - 6s 6s/step - loss: 48.0242 - costs: 44.0000
Epoch 7/7
1/1 [==============================] - 11s 11s/step - loss: 79.5336 - costs: 37.2500
Epoch 8/8
1/1 [==============================] - 7s 7s/step - loss: 39.0464 - costs: 32.8800
Epoch 9/9
1/1 [==============================] - 8s 8s/step - loss: 46.7698 - costs: 28.9100
Epoch 10/10
1/1 [==============================] - 4s 4s/step - loss: 195.7592 - costs: 35.5600
Epoch 11/11
1/1 [=========


KeyboardInterrupt



In [18]:
ep=540
for i in range(30):#range(50)
    zipped_list = list(zip(x_train_list,y_train_list))
    random.shuffle(zipped_list)
    x_train_list, y_train_list = zip(*zipped_list)
    for j in range(len(x_train_list)):
        history = Transformer.fit(x_train_list[j], y_train_list[j], shuffle=True, batch_size=128000, epochs=ep+1, initial_epoch=ep) #validation_data=(x_val_list[j],y_val_list[j])) #callbacks=[my_val_callback]
        ep = history.epoch[-1]+1

Epoch 541/541
1/1 [==============================] - 3s 3s/step - loss: 138.7464 - costs: 17.6400
Epoch 542/542
1/1 [==============================] - 7s 7s/step - loss: 25.9888 - costs: 7.6300
Epoch 543/543
1/1 [==============================] - 8s 8s/step - loss: 28.2314 - costs: 7.2600
Epoch 544/544
1/1 [==============================] - 4s 4s/step - loss: 50.2624 - costs: 4.7400
Epoch 545/545
1/1 [==============================] - 2s 2s/step - loss: 14.3631 - costs: 4.0200
Epoch 546/546
1/1 [==============================] - 3s 3s/step - loss: 21.5952 - costs: 9.5900
Epoch 547/547
1/1 [==============================] - 8s 8s/step - loss: 58.1834 - costs: 13.0900
Epoch 548/548
1/1 [==============================] - 3s 3s/step - loss: 178.6768 - costs: 18.1900
Epoch 549/549
1/1 [==============================] - 6s 6s/step - loss: 14.5614 - costs: 3.9500
Epoch 550/550
1/1 [==============================] - 6s 6s/step - loss: 35.1904 - costs: 7.6000
Epoch 551/551
1/1 [================

1/1 [==============================] - 3s 3s/step - loss: 15.6411 - costs: 6.2900
Epoch 627/627
1/1 [==============================] - 3s 3s/step - loss: 11.1904 - costs: 4.5600
Epoch 628/628
1/1 [==============================] - 4s 4s/step - loss: 45.7904 - costs: 6.2800
Epoch 629/629
1/1 [==============================] - 5s 5s/step - loss: 15.2196 - costs: 5.2200
Epoch 630/630
1/1 [==============================] - 7s 7s/step - loss: 31.8550 - costs: 8.0000
Epoch 631/631
1/1 [==============================] - 2s 2s/step - loss: 9.1692 - costs: 3.1000
Epoch 632/632
1/1 [==============================] - 6s 6s/step - loss: 19.5708 - costs: 4.9200
Epoch 633/633
1/1 [==============================] - 3s 3s/step - loss: 59.6125 - costs: 4.3900
Epoch 634/634
1/1 [==============================] - 5s 5s/step - loss: 14.2479 - costs: 4.8000
Epoch 635/635
1/1 [==============================] - 4s 4s/step - loss: 42.7916 - costs: 6.0200
Epoch 636/636
1/1 [==============================] - 5s

1/1 [==============================] - 6s 6s/step - loss: 16.7523 - costs: 2.3300
Epoch 712/712
1/1 [==============================] - 6s 6s/step - loss: 26.9532 - costs: 7.4100
Epoch 713/713
1/1 [==============================] - 5s 5s/step - loss: 21.0432 - costs: 7.0400
Epoch 714/714
1/1 [==============================] - 3s 3s/step - loss: 53.5315 - costs: 3.1000
Epoch 715/715
1/1 [==============================] - 5s 5s/step - loss: 13.6234 - costs: 4.5800
Epoch 716/716
1/1 [==============================] - 6s 6s/step - loss: 24.0458 - costs: 5.1000
Epoch 717/717
1/1 [==============================] - 6s 6s/step - loss: 17.3013 - costs: 4.7300
Epoch 718/718
1/1 [==============================] - 5s 5s/step - loss: 13.2413 - costs: 2.7200
Epoch 719/719
1/1 [==============================] - 3s 3s/step - loss: 14.3434 - costs: 5.2600
Epoch 720/720
1/1 [==============================] - 5s 5s/step - loss: 68.2722 - costs: 8.2600
Epoch 721/721
1/1 [==============================] - 5

1/1 [==============================] - 13s 13s/step - loss: 8.5583 - costs: 2.3400
Epoch 797/797
1/1 [==============================] - 5s 5s/step - loss: 24.3200 - costs: 6.1900
Epoch 798/798
1/1 [==============================] - 5s 5s/step - loss: 15.9632 - costs: 4.4700
Epoch 799/799
1/1 [==============================] - 3s 3s/step - loss: 10.5682 - costs: 4.2200
Epoch 800/800
1/1 [==============================] - 5s 5s/step - loss: 87.4570 - costs: 8.4500
Epoch 801/801
1/1 [==============================] - 3s 3s/step - loss: 53.4475 - costs: 4.0800
Epoch 802/802
1/1 [==============================] - 4s 4s/step - loss: 107.6719 - costs: 8.6600
Epoch 803/803
1/1 [==============================] - 8s 8s/step - loss: 29.7935 - costs: 4.3000
Epoch 804/804
1/1 [==============================] - 5s 5s/step - loss: 9.9052 - costs: 1.8300
Epoch 805/805
1/1 [==============================] - 4s 4s/step - loss: 43.5778 - costs: 3.0100
Epoch 806/806
1/1 [==============================] - 

1/1 [==============================] - 3s 3s/step - loss: 84.4239 - costs: 6.6900
Epoch 882/882
1/1 [==============================] - 13s 13s/step - loss: 23.3881 - costs: 5.3500
Epoch 883/883
1/1 [==============================] - 3s 3s/step - loss: 50.8061 - costs: 4.2900
Epoch 884/884
1/1 [==============================] - 4s 4s/step - loss: 26.1705 - costs: 3.6700
Epoch 885/885
1/1 [==============================] - 13s 13s/step - loss: 26.5652 - costs: 5.3800
Epoch 886/886
1/1 [==============================] - 6s 6s/step - loss: 18.6089 - costs: 4.8900
Epoch 887/887
1/1 [==============================] - 4s 4s/step - loss: 27.0819 - costs: 1.9900
Epoch 888/888
1/1 [==============================] - 5s 5s/step - loss: 14.7501 - costs: 4.7300
Epoch 889/889
1/1 [==============================] - 12s 12s/step - loss: 18.0167 - costs: 3.4700
Epoch 890/890
1/1 [==============================] - 11s 11s/step - loss: 12.2125 - costs: 2.1200
Epoch 891/891
1/1 [===========================

1/1 [==============================] - 3s 3s/step - loss: 108.9561 - costs: 7.8200
Epoch 967/967
1/1 [==============================] - 3s 3s/step - loss: 13.0526 - costs: 4.2800
Epoch 968/968
1/1 [==============================] - 4s 4s/step - loss: 29.9185 - costs: 2.0100
Epoch 969/969
1/1 [==============================] - 12s 12s/step - loss: 14.6671 - costs: 1.8400
Epoch 970/970
1/1 [==============================] - 6s 6s/step - loss: 9.4949 - costs: 1.7600
Epoch 971/971
1/1 [==============================] - 2s 2s/step - loss: 6.4138 - costs: 1.6800
Epoch 972/972
1/1 [==============================] - 5s 5s/step - loss: 41.3253 - costs: 4.9100
Epoch 973/973
1/1 [==============================] - 5s 5s/step - loss: 19.9614 - costs: 4.9900
Epoch 974/974
1/1 [==============================] - 13s 13s/step - loss: 29.9819 - costs: 4.7300
Epoch 975/975
1/1 [==============================] - 5s 5s/step - loss: 6.0532 - costs: 2.2000
Epoch 976/976
1/1 [==============================] -

1/1 [==============================] - 7s 7s/step - loss: 11.5562 - costs: 3.2800
Epoch 1051/1051
1/1 [==============================] - 5s 5s/step - loss: 13.1451 - costs: 4.1100
Epoch 1052/1052
1/1 [==============================] - 3s 3s/step - loss: 45.8003 - costs: 3.5100
Epoch 1053/1053
1/1 [==============================] - 11s 11s/step - loss: 25.9758 - costs: 1.7800
Epoch 1054/1054
1/1 [==============================] - 5s 5s/step - loss: 10.3761 - costs: 3.1500
Epoch 1055/1055
1/1 [==============================] - 7s 7s/step - loss: 16.0484 - costs: 3.1300
Epoch 1056/1056
1/1 [==============================] - 3s 3s/step - loss: 8.7658 - costs: 1.8000
Epoch 1057/1057
1/1 [==============================] - 13s 13s/step - loss: 12.4681 - costs: 1.8900
Epoch 1058/1058
1/1 [==============================] - 6s 6s/step - loss: 17.9035 - costs: 4.5900
Epoch 1059/1059
1/1 [==============================] - 3s 3s/step - loss: 7.5363 - costs: 2.5900
Epoch 1060/1060
1/1 [=============

In [ ]:
ep=5 
for i in range(24):#range(50)
    zipped_list = list(zip(x_train_list,y_train_list))
    random.shuffle(zipped_list)
    x_train_list, y_train_list = zip(*zipped_list)
    for j in range(len(x_train_list)):
        history = Transformer.fit(x_train_list[j], y_train_list[j], shuffle=True, batch_size=128, epochs=ep+1, initial_epoch=ep) #validation_data=(x_val_list[j],y_val_list[j])) #callbacks=[my_val_callback]
        ep = history.epoch[-1]+1

Epoch 6/6
413/413 [==============================] - 234s 565ms/step - loss: 8.5292 - costs: 1.5604
Epoch 7/7
79/79 [==============================] - 62s 782ms/step - loss: 3.7060 - costs: 1.2265
Epoch 8/8
313/313 [==============================] - 131s 419ms/step - loss: 12.5189 - costs: 0.9045
Epoch 9/9
385/385 [==============================] - 189s 492ms/step - loss: 10.3162 - costs: 1.2449
Epoch 10/10
79/79 [==============================] - 60s 755ms/step - loss: 3.0238 - costs: 0.9409
Epoch 11/11
383/383 [==============================] - 185s 482ms/step - loss: 3.9567 - costs: 0.4766
Epoch 12/12
79/79 [==============================] - 60s 759ms/step - loss: 2.9786 - costs: 0.9118
Epoch 13/13
278/278 [==============================] - 189s 679ms/step - loss: 1.4638 - costs: 0.4535
Epoch 14/14
313/313 [==============================] - 131s 418ms/step - loss: 21.9635 - costs: 1.5540
Epoch 15/15
210/210 [==============================] - 146s 696ms/step - loss: 4.8636 - costs: 1

79/79 [==============================] - 49s 621ms/step - loss: 3.2808 - costs: 1.0372
Epoch 166/166
383/383 [==============================] - 151s 394ms/step - loss: 2.8015 - costs: 0.3399
Epoch 167/167
313/313 [==============================] - 107s 341ms/step - loss: 4.2458 - costs: 0.3295
Epoch 168/168
408/408 [==============================] - 206s 504ms/step - loss: 3.1109 - costs: 0.8899
Epoch 169/169
413/413 [==============================] - 186s 450ms/step - loss: 6.0580 - costs: 1.1627
Epoch 170/170
313/313 [==============================] - 106s 340ms/step - loss: 14.8884 - costs: 1.1381
Epoch 171/171
261/261 [==============================] - 146s 560ms/step - loss: 2.9881 - costs: 0.9434
Epoch 172/172
416/416 [==============================] - 188s 452ms/step - loss: 1.7574 - costs: 0.3421
Epoch 173/173
313/313 [==============================] - 106s 338ms/step - loss: 10.0713 - costs: 0.7846
Epoch 174/174
313/313 [==============================] - 107s 342ms/step - loss

In [15]:
for j in range(len(x_val_list)):
    print(str(j//3+3), " Jobs ", str(j%3+2), " Machines:")
    performance = Transformer.evaluate(x_val_list[j], y_val_list[j])

3  Jobs  2  Machines:
1250/1250 [==============================] - 342s 274ms/step - loss: 44.1458 - costs: 1.1294
3  Jobs  3  Machines:
1250/1250 [==============================] - 350s 280ms/step - loss: 131.0477 - costs: 4.8325
3  Jobs  4  Machines:
1250/1250 [==============================] - 332s 265ms/step - loss: 212.6674 - costs: 9.2144
4  Jobs  2  Machines:
1563/1563 [==============================] - 463s 296ms/step - loss: 13.8030 - costs: 0.4894
4  Jobs  3  Machines:
1563/1563 [==============================] - 451s 288ms/step - loss: 52.7525 - costs: 2.2059
4  Jobs  4  Machines:
1563/1563 [==============================] - 461s 295ms/step - loss: 94.6736 - costs: 5.1114
5  Jobs  2  Machines:
1875/1875 [==============================] - 607s 324ms/step - loss: 5.4297 - costs: 0.3785
5  Jobs  3  Machines:
1875/1875 [==============================] - 613s 327ms/step - loss: 25.4819 - costs: 1.4054
5  Jobs  4  Machines:
1875/1875 [==============================] - 643s 343ms/s

In [17]:
Transformer.save('Transformer_no_batches.h5')

In [14]:
Transformer = keras.models.load_model('Transformer.h5', custom_objects={'FeedForward': FeedForward, 'Pointer': Pointer, 'TransformerLayer': TransformerLayer, 'Merge': Merge, 'AddZeroLine': AddZeroLine, 'MSE_with_Softmax': MSE_with_Softmax, 'costs':costs})
Transformer.run_eagerly = True